In [3]:
from bs4 import BeautifulSoup as bs
import urllib
import re
import string
import pandas as pd
from operator import itemgetter

In [4]:
def url(artist):
    base = "http://www.lyricsfreak.com/"
    name = artist.lower().strip().translate(None, string.punctuation).split(" ")
    if '' in name:
        name.remove('')
    init = name[0][0]
    search_name = "+".join(name)
    URL = base + init + "/" + search_name + "/"
    return URL

def fetch(artist):
    base = "http://www.lyricsfreak.com"
    song_list = []
    URL = url(artist)
    page = urllib.urlopen(URL)
    soup = bs(page, "lxml")
    songs = soup.findAll("table", attrs={"name": "song"})[0].findAll("tr")
    for s in songs:
        for n in s.findAll("td", attrs={"class": "colfirst"}):
            name = re.sub(" Lyrics","",n.find('a').attrs["title"])
            lyric_link = base + n.find('a').attrs["href"]
            song_list.append({name:lyric_link})
    return song_list

def fetch_all_artists(artist_list):
    lists = {}
    for i, artist in enumerate(artist_list):
        # print artist
        # print "\r{}%".format(100.0*(i+1)/len(artist_list)),
        song_list = []
        if artist[:3].lower() == 'the':
            try:
                song_list = fetch(artist)
            except IndexError:
                pass
            try:
                song_list += fetch(artist[4:])
            except IndexError:
                pass
        elif 'and' in artist.split(" "):
            try:
                song_list = fetch(artist)
            except IndexError:
                pass
            try:
                song_list += fetch(re.sub(' and ', ' & ', artist))
            except IndexError:
                pass
        else:
            try:
                song_list = fetch(artist)
            except:
                print "[Warning!] {}: Not Exist".format(artist)
        lists[artist] = song_list
    return lists

def ly_correct_url(URL):
    # input: lyric_page_url
    # output: lyric_correct_url
    page = urllib.urlopen(URL)
    soup = bs(page, 'lxml')
    base = 'http://www.lyricsfreak.com'
    try:
        correct_url = base + soup.find_all('a', attrs={'class': 'lb-correct'})[0].attrs['href']
    except IndexError:
        correct_url = None
    return correct_url

def fetch_lyrics(URL):
    # input: lyric_page_url
    URL2 = ly_correct_url(URL)
    if URL2:
        page = urllib.urlopen(URL2)
        soup = bs(page, 'lxml')
        lyric_text = soup.findAll('textarea', attrs={'name': 'lyrics'})[0].text
        return lyric_text
    else:
        return None

def fetch_one_artist_all_lyrics(artist):
    li =  [artist] #;print li
    d = fetch_all_artists(li) #;print d
    song_list = d[artist]
    lyrics_list = {} # {name: lyrics}
    for i, song in enumerate(song_list):
        # print '\r{}%'.format(100.0*(i+1)/len(song_list)),
        name = song.keys()[0] #;print name
        url = song.values()[0] #;print url
        lyrics = fetch_lyrics(url)
        lyrics_list[name] = lyrics
    return lyrics_list

def fetch_all_artists_all_lyrics(artist_list):
    artists_songs_lyrics = {} # {artist: {song: lyrics}}
    for i, artist in enumerate(artist_list):
        songs_lyrics = fetch_one_artist_all_lyrics(artist)
        artists_songs_lyrics[artist] = songs_lyrics
        print '\r{}%'.format(100.0*(i+1)/len(artist_list)),
    return artists_songs_lyrics

In [8]:
artist = ['Sir Douglas Quintet', 'The Royal Guardsmen', 'The Buckinghams', 'The Strangeloves', 'The Byrds']
res = fetch_all_artists_all_lyrics(artist)

100.0%


In [9]:
for k,v in res.iteritems():
    print k, len(v), '\b',
    num = 0
    for s, ly in v.iteritems():
        if ly != u'': # Not Exist!
            num += 1
    print num

The Buckinghams 9  6
The Royal Guardsmen 5  2
The Strangeloves 4  1
The Byrds 99  92
Sir Douglas Quintet 4  2


<br/><br/><br/><br/><br/><br/>

In [10]:
def dict2df(ly):
    df = pd.DataFrame(columns=['Artist', 'Song', 'Lyrics'])
    for a in ly:
        artist = a
        song_list = ly[a]
        if isinstance(song_list, dict):
            for s in song_list:
                song = s
                lyrics = song_list[s]
                record = pd.DataFrame.from_dict({'Artist': artist,
                                                 'Song': song,
                                                 'Lyrics': lyrics}
                                                ,orient='index').T
                df = df.append(record)  
    df = df.loc[:,['Artist', 'Song', 'Lyrics']]
    df = df.reset_index(drop=True)
    return df

In [11]:
AM_uncertain = dict2df(res)

In [21]:
AM_uncertain_with_ly = AM_uncertain[map(lambda x: False if x==u'' else True, AM_uncertain.Lyrics)]

In [23]:
len(AM_uncertain_with_ly)

103

In [24]:
AM_uncertain_with_ly.to_csv("AM_uncertain_with_ly.csv",encoding='utf-8', quoting=2)